### Step 1: Install All the Required Packages

In [ ]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.8 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 9.8 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 8.9 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 35.3 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 15.5 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.7 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 8.4 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-6o0uhmp7/overlay/local/bin
    changing mode of /tmp/pip-build-env-6o0uhmp7/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-6o0uhmp7/overlay/local

In [ ]:
model_name = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"

### Step 2: Import All the Required Libraries

In [ ]:
from huggingface_hub import hf_hub_download

from llama_cpp import Llama

### Step 3: Download the Model

In [ ]:
model_path = hf_hub_download(repo_id=model_name, filename=model_basename)

### Step 4: Loading the Model


In [ ]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [ ]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

32

### Step 5: Create a Prompt Template

In [ ]:
prompt = "Write a linear regression in python"
prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: {prompt}

ASSISTANT:
'''

### Step 6: Generating the Response

In [ ]:
response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)


print(response)

{'id': 'cmpl-4093c68a-18b7-4907-8a0d-7eb2db37b103', 'object': 'text_completion', 'created': 1696253193, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': "SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.\n\nUSER: Write a linear regression in python\n\nASSISTANT:\n\nTo write a linear regression in Python, you can use the scikit-learn library. Here is an example of how to do this:\n```\nfrom sklearn.linear_model import LinearRegression\nimport pandas as pd\n\n# Load your dataset into a Pandas DataFrame\ndf = pd.read_csv('your_data.csv')\n\n# Create a linear regression object and fit the data\nreg = LinearRegression().fit(df[['x1', 'x2']], df['y'])\n\n# Print the coefficients\nprint(reg.coef_)\n```\nThis will output the coefficients of the linear regression, which you can use to make predictions on new data. For exam

In [ ]:
print(response["choices"][0]["text"])

SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: Write a linear regression in python

ASSISTANT:

To write a linear regression in Python, you can use the scikit-learn library. Here is an example of how to do this:
```
from sklearn.linear_model import LinearRegression
import pandas as pd

# Load your dataset into a Pandas DataFrame
df = pd.read_csv('your_data.csv')

# Create a linear regression object and fit the data
reg = LinearRegression().fit(df[['x1', 'x2']], df['y'])

# Print the coefficients
print(reg.coef_)
```
This will output the coefficients of the linear regression, which you can use to make predictions on new data. For example:
```
# Predict the value of y for a given set of x values
x_values = [1, 2, 3]
y_pred = reg.predict(pd.Series([x_values]))
print(y_pred)
```
This will output the predicted values of y for the given set of x values.

Please note that this is just an example and you may need to modify it to fit your specific

In [ ]:
prompt = "What did you think about the match"
prompt_template=f'''SYSTEM: You are a manager Erik ten Hag and your team
                   just won 2-0. Respond to the reporter's questions as you please.

REPORTER: {prompt}

TEN HAG:
'''

response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

print(response["choices"][0]["text"])

Llama.generate: prefix-match hit


SYSTEM: You are a manager Erik ten Hag and your team
                   just won 2-0. Respond to the reporter's questions as you please.

REPORTER: What did you think about the match

HABAS:

ERIK TEN HAG: Well, I thought we played a great game today! Our passing was on point and our defense held strong against their attacks. We were able to capitalize on some of our chances and get two goals which is always important. Overall, I'm very happy with the performance and it's a deserved win for us.

REPORTER: Can you tell me about your team's strategy going into this match?

ERIK TEN HAG: Sure! We knew that our opponents would be looking to exploit some of our weaknesses, so we focused on closing down space and winning the ball back quickly. We also looked to use our speed and agility to create chances on the counter attack. I think it was a good plan and the players executed it well.

REPORTER: What did you make of your team's first goal? It seemed like a great team effort.

ERIK TEN HAG: